# Implementace domácího úkolu - rybí regrese 

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

**ÚKOL 1**: Načtěte data, vyhoďte přebytečné sloupce, vyberte vstupy a výstupy a připravte rozdělení na trénovací a testovací množinu.


- Načtěte si data pomocí pandas, vyberte požadované sloupce, které budete používat. 


In [ ]:
fish_data = pd.read_csv("fish_data.csv", index_col=0)
# fish_data = fish_data.drop(columns=[___])
fish_data

- Zvol si sloupec, který budeš používat jako odezvu (**Weight**). Do proměnné **X** ulož sloupce, které budeš používat jako příznaky, do proměnné **y** sloupec s odezvou.

*V teorii strojového učení se vstupy modelu (příznaky, vstupní proměnné) typicky označují písmenem X a výstupy písmenem y. Takto se často označují i proměnné v kódu. X představuje matici (neboli tabulku), kde každý řádek odpovídá jednomu datovému vzorku a každý sloupec jednomu příznaku (vstupní proměnné). y je vektor, neboli jeden sloupec s odezvou.*

In [ ]:
y = fish_data["Weight"]
X = fish_data.drop(columns=["Weight"])

- Rozděl data na trénovací a testovací. Všimni si, že máme v datech různé druhy ryb, na co si dát pozor? 

*Metoda  [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train%20test%20split#sklearn.model_selection.train_test_split)  nám data rozdělí náhodně na trénovací a testovací sadu. Velikost testovací množiny můžeme specifikovat parametrem test_size, jeho přednastavená (default) hodnota je 0.25, t. j. 25%.*

In [ ]:
from sklearn.model_selection import train_test_split 

X_train_raw, X_test_raw, y_train, y_test = ...

**ÚKOL 2**: Překódujte kategorické proměnné a přeškálujte všechny sloupce.

+ Překóduj potřebné sloupce pomocí OneHotEncoding. 

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

categorical_columns = ___ 

transformer = make_column_transformer(
    (_______, _____),
    remainder="passthrough"
)

X_train_onehot = transformer._______(X_train_raw)
X_test_onehot = transformer.________(X_test_raw)

pd.DataFrame(X_train_onehot, columns=transformer.get_feature_names_out())

+ Přeškáluj sloupce pomocí StandardScaler.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = ______
scaler._____(_____)

X_train = scaler.transform(X_train_onehot)
X_test = scaler.transform(X_test_onehot)

- Vyber si několik regresních modelů a zkus je použít. 

Pro dnešek možno zkusit:

  - [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) 
 
  - [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso)
      + hyperparametry: 
          * alpha, float, default=1.0 
 
- [SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)        
     + hyperparametry:
          * kernel, default rbf, one of ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’
          * C, float, optional (default=1.0)

# Odbočka: co jsou to ty *hyper-parametry*? 

U příkladů s černými krabičkami v první hodině jsme si (za vašimi zády) několikrát trochu pomohli a 
krabičce jsme předali na začátku nějaké parametry. Krabička totiž často umožňuje uživateli, aby si ji nakonfiguroval. V terminologii krabiček si můžeme představit, že krabička má na sobě různé páčky, kterými se dá seřídit. Těmito páčkami se nastavují tzv. **hyper-parametry** modelu. Všechny modely, které najdeš v knihovně Scikit-learn, mají nějaké výchozí nastavení a půjdou použít i bez toho, aby ses nastavením těchto hyper-parametrů zabývala.
V případě, že model nedává uspokojivý výsledek, můžeš zkusit tyto parametry upravit, např. vyzkoušet několik různých nastavení a porovnat hodnotu metriky na testovací množině.

U seznamu výše máme některé hyperparametry uvedené. Parametry často souvisejí s regularizací (výše *alpha*, *C*). **Regularizace** znamená, že model kromě toho, že se snaží nafitovat tak, aby odpovídal datům (dával správné odpovědi), zohledňuje nějaké další kriterium. Typicky toto kritérium hlídá, aby výstup modelu moc neosciloval, apod. Podobně jako jsi v příkladu s krajinou říkala, že řešení volíš tak, aby bylo *plynulé*, *hezké*, *odpovídalo obvyklým* krajinám. 

Proces výběru modelu včetně jeho parametrů se nazývá **model selection**, v knihovně Scikit-learn najdeš nástroje, které ti mohou pomoci, pod heslem [Model selection](https://scikit-learn.org/stable/model_selection.html).

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR

In [ ]:
model_zoo = {
    "linear_regression": LinearRegression(),
    "lasso_var1": Lasso(alpha=1.0),
    "lasso_var2": Lasso(alpha=1e-03),
    "SVR_rbf": SVR(kernel="rbf", C=1e04),
    "SVR_poly": SVR(kernel="poly",C=1e04),
}

+ K trénovaní (fitování) slouží metoda `fit`, k predikci pro nové vzory metoda `predict`. 
```
  model.fit(X_train, y_train)
  pred = model.predict(X_test)
```

+ Metriku nemusíš programovat, k dispozici máš `mean_absolute_error`, `mean_squared_error` a `r2_score`.
```
  metrika = mean_absolute_error(y_test, pred)
```  

**ÚKOL 3**: Dopište funkci `fit_and_eval` dle instrukcí.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def fit_and_eval(X_train, y_train, X_test, y_test, model, name):
    """ 1. Natrénuje model na trénovací množině.
        2. Spočte hodnoty metrik na trénovací i testovací množině.
        vrátí slovník ve tvaru {"název metriky": hodnota} 
    """ 
    # zde dopiš kód
    ... 
    return {
        "MSE_test": ____,
        "MSE_train": ____
    }

In [ ]:
results = []
for name, model in model_zoo.items(): 
    result = fit_and_eval(X_train, y_train, X_test, y_test, model, name)
    result["model"] = name
    results.append(result)
    
pd.DataFrame(results)

Naučili jsme několik modelů. Zamysli se teď na chvilku, který by sis vybrala a proč. 
Označme si jej jako `best_model`.  Můžeš si i zkusit pohrát s hyperparametry a zvolit jiné nastavení.

In [ ]:
# doplň jméno modelu, který jsi vybrala

best_model = model_zoo[____]

# A pozor, překvapení ...  další testovací množina

Data jsme si rozdělili na trénovací a testovací. Trénovací jsme použili na učení modelu. 
**Ale pozor!** Testovací množinu jsme použili k výběru modelu. Metrika na testovací množině nám tedy 
nedává nezávislý odhad toho, jak se bude náš model chovat na neznámých datech. Byl totiž vybrán tak, 
aby dával dobré výsledky na testovací množině. 

Testovací množina nám slouží jako odhad generalizačních schopností modelu. Neměla by ale být použita ani při učení,
ani při výběru modelu. Část, kterou si oddělíme na "testování" pro účely výběru modelu, nazýváme správně 
**validační** množina. 
**Pozor:** Pokud jsme ale tuto validační množinu použili k výběru modelu, nesmíme ji používat k samotnému hodnocení generalizačních schopností tohoto modelu. 

A proto teď přichází opravdová testovací data, načtěte je ze souboru `fish_data_test.csv`.

In [ ]:
test_data = pd.read_csv("fish_data_test.csv", index_col=0)

test_data.pop("ID")
y_real_test = test_data.pop("Weight")
X_real_test = test_data 

X_real_test_transformed = _____
X_real_test_scaled = _______

In [ ]:
y_pred_test = best_model.predict(X_real_test_scaled)

print(f"MAE {mean_absolute_error(y_real_test, y_pred_test):.3f}")
print(f"MSE {mean_squared_error(y_real_test, y_pred_test):.3f}")
print(f"R2  {r2_score(y_real_test, y_pred_test):.3f}")

In [ ]:
for weight, predicted_weight in zip(y_real_test, y_pred_test):
    print(f"{weight:>10.1f}     {predicted_weight:>10.1f}")

# Visualizace na závěr 

+ Pro představu si zobrazme závislost váhy ryby na délce Length3. Zobrazíme zvlášt pro různé druhy, tedy např. pro Bream a Roach.  

In [ ]:
is_bream = X_real_test["Species"] == "Bream"
predicted_bream_weights = best_model.predict(X_real_test_scaled[is_bream])

is_roach = X_real_test["Species"] == "Roach"
predicted_roach_weights = best_model.predict(X_real_test_scaled[is_roach])

In [ ]:
result_bream = pd.DataFrame()
result_bream["length"] = X_real_test[is_bream]["Length3"]
result_bream["weight"] = y_real_test[is_bream]
result_bream["predicted"] = predicted_bream_weights
result_bream = result_bream.sort_values("length")

result_roach = pd.DataFrame()
result_roach["length"] = X_real_test[is_roach]["Length3"]
result_roach["weight"] = y_real_test[is_roach]
result_roach["predicted"] = predicted_roach_weights
result_roach = result_roach.sort_values("length")

In [ ]:
import matplotlib.pyplot as plt 

fig, ax = plt.subplots(1, 2)

ax[0].plot(result_bream["length"], result_bream["weight"], label="true weight", marker="o");
ax[0].plot(result_bream["length"], result_bream["predicted"], label="prediction", marker="o");
ax[0].legend()
ax[0].set_title("Bream")

ax[1].plot(result_roach["length"], result_roach["weight"], label="true weight", marker="o");
ax[1].plot(result_roach["length"], result_roach["predicted"], label="prediction", marker="o");
ax[1].legend()
ax[1].set_title("Roach");